In [1]:
import requests
import pandas as pd
import json
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime, date, timedelta

In [2]:
# read historical csv and read to data frame
csv_name = 'historical_DOGE-USD.csv'
prices_df = pd.read_csv(csv_name)
#prices_df.head()

In [3]:
# get current price of doge and add overwrite todays entry
json_response = requests.get('https://sochain.com//api/v2/get_price/DOGE/USD').json()

i = 0
current_price = -1
if json_response['status'] == 'success':
  while current_price <= 0:
    current_price = float(json_response['data']['prices'][i]['price'])
    i += 1

#print(current_price)

In [4]:
# add today's price to the csv for next run. keep the highest price when duplicate dates
df_today = {'Date': date.today(), 'High': current_price}
prices_df = prices_df.append(df_today, ignore_index = True)
prices_df = prices_df.sort_values(['Date', 'High'] , ascending=True).drop_duplicates(['Date'] , keep='last').reset_index(drop=True)
prices_df.set_index(['Date']).to_csv(csv_name)

In [5]:
# accept date from user
calendar = widgets.DatePicker(
    description='Select Date',
    disabled=False
)
#calendar

In [6]:
# accept money invested on date from user
investment_input = widgets.BoundedFloatText(
    value=100.00,
    min=0,
    max=1_000_000_000_000,
    description='Investment $',
    disabled=False
)
#investment_input

In [7]:
# create a button for calculating return
button = widgets.Button(description="Calculate")
#button

In [8]:
# define labels used in function
select_date_label = widgets.Label('Select a valid date.')
price_not_found_label = widgets.Label('Price not found. Please select another date.')
price_on_date_label = widgets.Label()
coins_bought_label = widgets.Label()
current_price_label = widgets.Label()
current_value_label = widgets.Label()
percent_change_label = widgets.Label() 
wordy_label = widgets.Label() 

def on_click_calculate(change):
    # read calendar widget
    buy_date = calendar.value
    if buy_date is None:
        display(select_date_label)
    elif date.today() - buy_date <= timedelta(days=0):
        display(select_date_label)
    else:
        # hide date warning label
        select_date_label.layout.visibility = 'hidden'
        buy_date = str(calendar.value)
        
        try:
            # get price on day in calendar widget
            price_on_date = prices_df[prices_df['Date']==buy_date]['High'].values[0]
        except IndexError as e:
            display(price_not_found_label)
        else:
            price_not_found_label.layout.visibility = 'hidden'
            # read initial investment widget
            initial_investment = investment_input.value

            # calculate number of coins bought with initial investment
            coins_bought = initial_investment / price_on_date

            # calculate current value
            current_value = coins_bought * current_price

            # calculate percent returns
            percent_change = (current_value - initial_investment) / initial_investment * 100

            # display output (couldn't figure out how to do multi-line label)
            price_on_date_label.value = f"Price on date: ${round(price_on_date,2)}"
            coins_bought_label.value = f"Coins bought: {round(coins_bought,1)}"
            current_price_label.value = f"Current price: ${round(current_price,2)}"
            current_value_label.value = f"Current value: ${round(current_value,2)}"
            percent_change_label.value = f"Percent change: {round(percent_change,2)}%"
            wordy_label.value = f"On {buy_date}, 1 dogecoin was worth \${round(price_on_date,2)}.  With \${initial_investment}, you could have bought {round(coins_bought,1)} coins.  Today, that'd be worth \${round(current_value,2)} at \${round(current_price,2)} per coin.  That's a {round(percent_change,2)}% gain you missed out on!"

button.on_click(on_click_calculate)

In [9]:
widgets.VBox([widgets.HTML('<b>See how bad you missed out on dogecoin!'), 
              investment_input, calendar, button, 
              #price_on_date_label, 
              price_on_date_label, 
              current_price_label, 
              current_value_label, 
              percent_change_label,
              wordy_label])

In [10]:
# Uncomment to make into a voila app (replace the word "notebooks" in the URL with "voila/render")
!jupyter serverextension enable --sys-prefix voila



Enabling: voila
- Writing config: /home/jordan/anaconda3/etc/jupyter
    - Validating...
/home/jordan/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
      voila 0.2.10 OK


In [11]:
# TODO: Make WHAT-IF simulations for what-if buy price or what-if sell price